In [5]:
text = "안녕하세요 👋 (hello in Korean!)"
text

'안녕하세요 👋 (hello in Korean!)'

In [7]:
print([ord(c) for c in text])

[50504, 45397, 54616, 49464, 50836, 32, 128075, 32, 40, 104, 101, 108, 108, 111, 32, 105, 110, 32, 75, 111, 114, 101, 97, 110, 33, 41]


In [13]:
list(text.encode('utf-8'))

[236,
 149,
 136,
 235,
 133,
 149,
 237,
 149,
 152,
 236,
 132,
 184,
 236,
 154,
 148,
 32,
 240,
 159,
 145,
 139,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

In [65]:
with open('asimov.txt', 'r') as f:
    asimov = f.read()
asimov = asimov[:1000]
print(asimov[37:565])

HARI SELDON-... bom In the 1 1,988th year of the Galactic Era; died 12,069. The dates are 
more commonly given In terms of the current Foundational Era as - 79 to the year 1 F.E. Born 
to middle-class parents on Flelicon, Arcturus sector (where his father, In a legend of doubtful 
authenticity, was a tobacco grower in the hydroponic plants of the planet), he early showed 
amazing ability in mathematics. Anecdotes concerning his ability are innumerable, and some 
are contradictory. At the age of two, he is said to have ... 


In [74]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def replace_max_pair(tokens, stats, max_pair, id_to_swap):
    i = len(tokens) - 2
    while i >= 0:
        if tokens[i] == max_pair[0] and tokens[i+1] == max_pair[1]:
            tokens[i] = id_to_swap
            del tokens[i+1]
            i -= 1
        i -= 1
    return tokens

In [84]:
with open('asimov.txt', 'r') as f:
    asimov = f.read()
asimov = asimov[:300]
tokens = asimov.encode('utf-8')
tokens = list(map(int, tokens))


id_to_swap = 256
for i in range(100):
    stats = get_stats(tokens)
    max_pair, count = max(stats.items(), key=lambda x: x[1])
    if count < 3:
        print('count < 3')
        break
    print(f'[{id_to_swap} - {len(tokens)}] count: {count}, chr(c): {[chr(c) for c in max_pair]}')
    tokens = replace_max_pair(tokens, stats, max_pair, id_to_swap)
    id_to_swap += 1


[256 - 300] count: 8, chr(c): ['e', ' ']
[257 - 292] count: 7, chr(c): ['s', ' ']
[258 - 285] count: 6, chr(c): ['n', ' ']
[259 - 279] count: 6, chr(c): ['t', 'h']
[260 - 273] count: 4, chr(c): ['.', ' ']
[261 - 269] count: 4, chr(c): ['ă', 'Ā']
[262 - 265] count: 4, chr(c): ['a', 'r']
[263 - 261] count: 3, chr(c): [' ', 'I']
[264 - 258] count: 3, chr(c): ['\n', '\n']
[265 - 255] count: 3, chr(c): [' ', '1']
[266 - 252] count: 3, chr(c): [' ', 'ą']
[267 - 249] count: 3, chr(c): ['c', 't']
[268 - 246] count: 3, chr(c): ['o', 'r']
[269 - 243] count: 3, chr(c): ['o', 'n']
[270 - 240] count: 3, chr(c): ['e', 'r']
count < 3


In [111]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids


with open('asimov.txt', 'r') as f:
    asimov = f.read()
asimov = asimov[:600]
tokens = asimov.encode('utf-8')
tokens = list(map(int, tokens))

vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)

merges = {} # (int, int) -> int
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f'merging {pair} into new token {idx}')
    ids = merge(ids, pair, idx)
    merges[pair] = idx


merging (101, 32) into new token 256
merging (115, 32) into new token 257
merging (32, 116) into new token 258
merging (104, 256) into new token 259
merging (46, 32) into new token 260
merging (111, 110) into new token 261
merging (258, 259) into new token 262
merging (97, 114) into new token 263
merging (105, 99) into new token 264
merging (97, 116) into new token 265
merging (101, 114) into new token 266
merging (44, 32) into new token 267
merging (32, 10) into new token 268
merging (111, 102) into new token 269
merging (121, 32) into new token 270
merging (101, 110) into new token 271
merging (105, 110) into new token 272
merging (108, 97) into new token 273
merging (111, 114) into new token 274
merging (108, 101) into new token 275


In [112]:
print(f'len(tokens): {len(tokens)} -> {len(ids)}')
print(f'compression ratio: {len(tokens) / len(ids):.2f}x')

len(tokens): 600 -> 456
compression ratio: 1.32x


In [113]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    # Give ids (list of ints), return a string
    tokens = b"".join([vocab[idx] for idx in ids])
    text = tokens.decode('utf-8', errors='replace')
    return text

print(decode(ids))

PART I 

THE PSYCHOHISTORIANS 

i. 

HARI SELDON-... bom In the 1 1,988th year of the Galactic Era; died 12,069. The dates are 
more commonly given In terms of the current Foundational Era as - 79 to the year 1 F.E. Born 
to middle-class parents on Flelicon, Arcturus sector (where his father, In a legend of doubtful 
authenticity, was a tobacco grower in the hydroponic plants of the planet), he early showed 
amazing ability in mathematics. Anecdotes concerning his ability are innumerable, and some 
are contradictory. At the age of two, he is said to have ... 

... Undoubtedly his greatest cont


In [115]:
merges

{(101, 32): 256,
 (115, 32): 257,
 (32, 116): 258,
 (104, 256): 259,
 (46, 32): 260,
 (111, 110): 261,
 (258, 259): 262,
 (97, 114): 263,
 (105, 99): 264,
 (97, 116): 265,
 (101, 114): 266,
 (44, 32): 267,
 (32, 10): 268,
 (111, 102): 269,
 (121, 32): 270,
 (101, 110): 271,
 (105, 110): 272,
 (108, 97): 273,
 (111, 114): 274,
 (108, 101): 275}

In [124]:
def encode(text):
    # Give a string, return a list of ints
    tokens = list(text.encode('utf-8'))
    while len(tokens) > 1:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float('inf')))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode('Hello Hari Seldon!'))

[72, 101, 108, 108, 111, 32, 72, 263, 105, 32, 83, 101, 108, 100, 261, 33]


In [128]:
import regex
gpt2pat = regex.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(regex.findall(gpt2pat, 'Hello Hari Seldon!'))

['Hello', ' Hari', ' Seldon', '!']


In [129]:
import tiktoken

ModuleNotFoundError: No module named 'tiktoken'

In [130]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.3 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 KB 22.2 MB/s eta 0:00:00


In [138]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")
tokenizer.encode("Hello Hari Seldon!")

[9906, 98545, 328, 56143, 0]

In [139]:
tokenizer.decode(tokenizer.encode("Hello Hari Seldon!"))

'Hello Hari Seldon!'